In [1]:
%matplotlib inline

In [2]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plot

In [35]:
resp = pd.read_csv('sampleData/download.csv', header=None)
resp.columns = ['time', 'resp', 'tone']

In [36]:
bci = pd.read_csv('sampleData/OpenBCI-RAW-2016-12-30_18-07-39.txt', sep=', ', skiprows=4, header=None)
bci.columns = ['tix'] + [str(i) for i in range(1,9)] + ['a', 'b', 'c', 'time']

/home/mike/ve/ml/lib/python3.5/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


In [37]:
resp.sort_values('time', inplace=True)

In [38]:
t0 = bci.loc[0, 'time'] 
tmid_bci = bci.loc[len(bci) // 2, 'time']
tmid_resp = resp.loc[len(resp) // 2, 'time']
delta_t = tmid_resp-tmid_bci
print(tmid_bci, tmid_resp, delta_t, t0)

resp['time'] += tmid_bci-tmid_resp # stupid hack to get the times to line up right
resp['time'] -= t0 
bci['time'] -= t0

1.48313927979e+12 1484171949901 1032670109.0 1.48313927268e+12


In [43]:
tonef = resp['tone'].replace({'C4':1, 'C5':2, 'C3': 0})
resp['tonef'] = pd.Series(tonef, dtype='float64')
# resp['tonef'].dtype

In [44]:
resp.dtypes

time     float64
resp       int64
tone      object
tonef    float64
dtype: object

In [45]:
resp

,time,resp,tone,tonef
0,6050.0,0,C4,1.0
1,6444.0,0,C4,1.0
2,6844.0,0,C4,1.0
7,7114.0,2,0,0.0
3,7244.0,0,C4,1.0
8,7366.0,2,0,0.0
9,7608.0,2,0,0.0
4,7644.0,0,C5,2.0
10,7836.0,1,0,0.0
5,8044.0,0,C4,1.0


In [46]:
len(bci)

3563

In [48]:
resp['time'] = pd.Series(resp['time'], dtype='int64') # Converting to int to allow for indexing safely
bci['time'] = pd.Series(bci['time'], dtype='int64')

In [49]:
bci

,tix,1,2,3,4,5,6,7,8,a,b,c,time
0,0,-120122.36,-113415.72,-114035.63,-117027.27,-136190.16,-148000.78,-155031.11,-131657.89,0.01,0.10,1.03,0
1,1,-120122.92,-113416.37,-114035.91,-117027.36,-136190.20,-148000.81,-155031.06,-131657.52,0.00,0.00,0.00,4
2,2,-120119.12,-113412.79,-114034.35,-117025.66,-136188.48,-147998.63,-155028.94,-131653.63,0.00,0.00,0.00,8
3,3,-120115.72,-113409.64,-114032.02,-117023.45,-136186.20,-147996.28,-155026.94,-131651.13,0.00,0.00,0.00,12
4,4,-120121.53,-113415.52,-114036.18,-117027.72,-136190.31,-148001.00,-155031.83,-131657.11,0.00,0.00,0.00,16
5,5,-120124.88,-113418.91,-114038.69,-117030.06,-136192.45,-148003.45,-155034.17,-131659.50,0.00,0.00,0.00,19
6,6,-120117.47,-113411.83,-114033.43,-117024.61,-136186.97,-147997.47,-155028.16,-131651.81,0.00,0.00,0.00,24
7,7,-120113.48,-113408.26,-114031.09,-117022.48,-136184.84,-147994.97,-155025.97,-131648.81,0.01,0.12,1.03,27
8,8,-120118.41,-113413.24,-114034.55,-117026.17,-136188.38,-147999.03,-155030.33,-131654.23,0.00,0.00,0.00,32
9,9,-120119.88,-113414.83,-114034.80,-117026.33,-136188.44,-147999.41,-155030.70,-131654.92,0.00,0.00,0.00,37


In [50]:
mid = len(bci) // 2
bci2 = bci.iloc[mid-100:mid+100]

In [51]:
# bci2.merge(resp, 'outer', on='time', sort=True)

In [52]:
# bci.merge(resp, 'outer', on='time', sort=True).to_csv('testmerge.csv')
mframe = pd.merge_asof(bci, resp, on='time', allow_exact_matches=False)

In [53]:
mframe

,tix,1,2,3,4,5,6,7,8,a,b,c,time,resp,tone,tonef
0,0,-120122.36,-113415.72,-114035.63,-117027.27,-136190.16,-148000.78,-155031.11,-131657.89,0.01,0.10,1.03,0,NaN,NaN,NaN
1,1,-120122.92,-113416.37,-114035.91,-117027.36,-136190.20,-148000.81,-155031.06,-131657.52,0.00,0.00,0.00,4,NaN,NaN,NaN
2,2,-120119.12,-113412.79,-114034.35,-117025.66,-136188.48,-147998.63,-155028.94,-131653.63,0.00,0.00,0.00,8,NaN,NaN,NaN
3,3,-120115.72,-113409.64,-114032.02,-117023.45,-136186.20,-147996.28,-155026.94,-131651.13,0.00,0.00,0.00,12,NaN,NaN,NaN
4,4,-120121.53,-113415.52,-114036.18,-117027.72,-136190.31,-148001.00,-155031.83,-131657.11,0.00,0.00,0.00,16,NaN,NaN,NaN
5,5,-120124.88,-113418.91,-114038.69,-117030.06,-136192.45,-148003.45,-155034.17,-131659.50,0.00,0.00,0.00,19,NaN,NaN,NaN
6,6,-120117.47,-113411.83,-114033.43,-117024.61,-136186.97,-147997.47,-155028.16,-131651.81,0.00,0.00,0.00,24,NaN,NaN,NaN
7,7,-120113.48,-113408.26,-114031.09,-117022.48,-136184.84,-147994.97,-155025.97,-131648.81,0.01,0.12,1.03,27,NaN,NaN,NaN
8,8,-120118.41,-113413.24,-114034.55,-117026.17,-136188.38,-147999.03,-155030.33,-131654.23,0.00,0.00,0.00,32,NaN,NaN,NaN
9,9,-120119.88,-113414.83,-114034.80,-117026.33,-136188.44,-147999.41,-155030.70,-131654.92,0.00,0.00,0.00,37,NaN,NaN,NaN


In [58]:
mframe['dtone'] = mframe['tonef'].diff().fillna(0)
mframe['oddball'] = mframe['dtone'] == 2
mframe.drop(['a', 'b', 'c', 'dtone'], axis=1, inplace=True)

In [59]:
mframe.to_csv('testmerge.csv')
print(mframe.shape, bci.shape)

(3563, 14) (3563, 13)


In [57]:
resp['tone'].dtype

dtype('O')

In [72]:
import os
orig = os.getcwd()

In [73]:
os.getcwd()
os.path.dirname(os.getcwd())
os.path.split(os.getcwd())

('/home/mike/w/bci_erp', 'python')

In [74]:
os.chdir('..')

In [75]:
os.path.split(os.getcwd())

('/home/mike/w', 'bci_erp')